In [ ]:
%%capture

%load_ext autoreload
%autoreload 2
%matplotlib inline
%load_ext tfl_training_anomaly_detection

In [ ]:
%presentation_style

In [ ]:
%%capture

%set_random_seed 12

In [ ]:
%load_latex_macros

# Anomaly Detection via Isolation
<img src="_static/images/aai_presentation_first_slide.svg" alt="Snow" style="width:100%;">

In [ ]:
import numpy as np
import itertools as it
from tqdm import tqdm

import matplotlib
from matplotlib import pyplot as plt
import plotly.express as px
import pandas as pd

import ipywidgets as widgets

from tfl_training_anomaly_detection.exercise_tools import evaluate, get_kdd_data, get_house_prices_data, create_distributions, contamination, \
perform_rkde_experiment, get_mnist_data

from ipywidgets import interact

from sklearn.metrics import roc_auc_score, average_precision_score
from sklearn.model_selection import RandomizedSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelBinarizer
from sklearn.ensemble import IsolationForest
from sklearn import metrics
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.neighbors import KernelDensity

from tfl_training_anomaly_detection.vae import VAE, build_decoder_mnist, build_encoder_minst, build_contaminated_minst

from tensorflow import keras

%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (5, 5)


# Anomaly Detection via Isolation
**Idea:** An anomaly should allow  "simple" descriptions that distinguish it from the rest of the data.

- Descriptions: Conjunction of single attribute tests, i.e.
  $X_i \leq c$ or $X_i > c$.
- Example: $X_1 \leq 1.2 \text{ and } X_5 > -3.4 \text{ and }	X_7 \leq 5.6$.
- Complexity of description: Number of conjunctions.

Moreover, we assume that a short random descriptions will have a significantly larger chance of isolating an anomaly
than isolating any nominal point.

- Choose random isolating descriptions and compute anomaly score from average complexity.

## Isolation Tree
Isolation Forest (iForest) implements this idea by generating an ensemble of random decision trees.
Each tree is built as follows:


**Input:** Data set (subsample) $X$, maximal height $h$
- Randomly choose feature $i$ and split value $s$ (in range of data)
- Recursively build subtrees on $X_L = \{x\in X\mid x_i \leq s\}$ and $X_R = X\setminus X_L$
- Stop if remaining data set $ \leq 1$ or maximal height reached
- Store test $x_i\leq s$ for inner nodes and $|X|$ for leaf nodes



## Visualization
<center>
<img src="_static/images/partition.png" align="center" width="400">
    Isolation Tree as Partition Diagram
</center>

## Isolation Depth
Depth of an observation $x$ in an isolation tree is defined as the expected number of tests needed to isolate $x$.


**Input:** Observation $x$
- ${\ell} = $ length of path from root to leaf according to tests
- ${n} = $ size of remaining data set in leaf node
- ${c(n)} =$ expected length of a path in a BST with $n$ nodes $={O}(\log n)$
- ${h(x)} = \ell + c(n)$



<table>
    <tr>
        <td style="background-color:#FFFFFF;"><img src="_static/images/blob.png" width="300"></td>
        <td style="background-color:#FFFFFF;"><img src="_static/images/tree.png"  width="300"></td>
    </tr>
    <tr>
        <td colspan="2" style="background-color:#FFFFFF;"><center>Isolation Depth of Outlier (red) and nominal (blue)</center></td>
    </tr>
</table>


## Isolation Forest
- Train $k$ isolation trees on subsamples of size $N$


<table>
    <tr>
        <td style="background-color:#FFFFFF;"><img src="_static/images/inrad.png" width="400"></td>
        <td style="background-color:#FFFFFF;"><img src="_static/images/outrad.png"  width="400"></td>
    </tr>
    <tr>
        <td colspan="2"><center>Isolation depth of nominal point (left) and outlier (right)</center></td>
    </tr>
</table>

# Variants of Isolation Forest

## Variant: Random Robust Cut Forest
**New Rule to Choose Split Test:**
- $\ell_i$: length of the $i$th component of the bounding box around current data set
- Choose dimension $i$ with probability $\frac{\ell_i}{\sum_j \ell_j}$
- More robust against "noise dimensions"

<center>
<img src="_static/images/partition2.png" align="center" width="400" caption="Comparisson IForest and RRCF">
</center>

## Variant: Extended Isolation Forest
**New split criterion:**
- Uniformly choose a normal and an orthogonal hyperplane through the data
- Removes a bias that was empirically observed when plotting the outlier score of iForest on low dimensional data sets

<center>
<img src="_static/images/extended.png" align="center" width="80%">
</center>

# Exercise: Network Security

In the final exercise of today you will have to develop an anomaly detection system for network traffic.

## Briefing
A large e-commerce company __A__ is experiencing downtime due to attacks on their infrastructure.
You were instructed to develop a system that can detect malicious connections to the infrastructure.
It is planned that suspicious clients will be banned.

Another data science team already prepared the connection data of the last year for you. They also separated a test set and manually identified and labeled attacks in that data.

## The Data
We will work on a version of the classic KDD99 data set.

### Kddcup 99 Data Set
======================

The KDD Cup '99 dataset was created by processing the tcp dump portions
of the 1998 DARPA Intrusion Detection System (IDS) Evaluation dataset,
created by MIT Lincoln Lab [1]. The artificial data (described on the `dataset's
homepage <https://kdd.ics.uci.edu/databases/kddcup99/kddcup99.html>`_) was
generated using a closed network and hand-injected attacks to produce a
large number of different types of attack with normal activity in the
background.

    =========================   ====================================================
    Samples total               976158
    Dimensionality              41
    Features                    string (str), discrete (int), continuous (float)
    Targets                     str, 'normal.' or name of the anomaly type
    Proportion of Anomalies     1%
    =========================   ====================================================


## Task
You will have to develop the system on your own. In particular, you will have to
- Explore the data.
- Choose an algorithm.
- Find a good detection threshold.
- Evaluate and summarize your results.
- Estimate how much __A__ could save through the use of your system.

In [ ]:
X_train,X_test,y_test = get_kdd_data()

# Explore Data

In [ ]:
#
# Add your exploration code
#
X_train = pd.DataFrame(X_train)
X_test = pd.DataFrame(X_test)

In [ ]:
# get description
X_train.describe()

In [ ]:
# get better description
X_train.drop(columns=[1,2,3]).astype(float).describe()

In [ ]:
# Check for NaNs
print("Number of NaNs: {}".format(X_train.isna().sum().sum()))

In [ ]:
#
# Add your preperation code here
#

# Encode string features
binarizer = LabelBinarizer()
one_hots = None
one_hots_test = None
for i in [1, 2, 3]:
    binarizer.fit(X_train[[i]].astype(str))
    if one_hots is None:
        one_hots = binarizer.transform(X_train[[i]].astype(str))
        one_hots_test = binarizer.transform(X_test[[i]].astype(str))
    else:
        one_hots = np.concatenate([one_hots, binarizer.transform(X_train[[i]].astype(str))], axis=1)
        one_hots_test = np.concatenate([one_hots_test, binarizer.transform(X_test[[i]].astype(str))], axis=1)

X_train.drop(columns=[1,2,3], inplace=True)
X_train_onehot = pd.DataFrame(np.concatenate([X_train.values, one_hots], axis=1))

X_test.drop(columns=[1,2,3], inplace=True)
X_test_onehot = pd.DataFrame(np.concatenate([X_test.values, one_hots_test], axis=1))

In [ ]:
# Encode y
y_test_bin = np.where(y_test == b'normal.', 0, 1)


In [ ]:
# Remove suspicious data
# This step is not strictly neccessary but can improve performance
suspicious = X_train_onehot.apply(lambda col: (col - col.mean()).abs() > 4 * col.std() if col.std() > 1 else False)
suspicious = suspicious.any(axis=1)# 4 sigma rule
print('filtering {} suspicious data points'.format(suspicious.sum()))
X_train_clean = X_train_onehot[~suspicious]

# Summary
- Isolation Forest empirically shows very good performance up to relatively high dimensions
- It is relatively robust against contamination
- Usually little need for hyperparameter tuning

## Implementations
- Sklearn: [sklearn.ensemble.IsolationForest](https://scikit-learn.org/stable/modules/generated/sklearn.ensemble.IsolationForest.html)
- Extended Isolation Forest: [variant](https://github.com/sahandha/eif)
- Random Robust Cut Forest: [variant](https://github.com/kLabUM/rrcf)

# Choose Algorithm

In [ ]:
# TODO: implement proper model selection
iforest = IsolationForest()
iforest.fit(X_train_clean)

In [ ]:
# find best threshold
X_test_onehot, X_val_onehot, y_test_bin, y_val_bin = train_test_split(X_test_onehot, y_test_bin, test_size=.5)
y_score = -iforest.score_samples(X_val_onehot).reshape(-1)

# Evaluate Solution

In [ ]:
#
# Insert evaluation code
#

# calculate scores if any anomaly is present
if np.any(y_val_bin == 1):
    eval = evaluate(y_val_bin, y_score)
    prec, rec, thr = eval['PR']
    f1s = 2 * (prec * rec)/(prec + rec)
    threshold = thr[np.argmax(f1s)]

    y_score = -iforest.score_samples(X_test_onehot).reshape(-1)
    y_pred = np.where(y_score < threshold, 0, 1)

    print('Precision: {}'.format(metrics.precision_score(y_test_bin, y_pred)))
    print('Recall: {}'.format(metrics.recall_score(y_test_bin, y_pred)))
    print('F1: {}'.format(metrics.f1_score(y_test_bin, y_pred)))

<img src="_static/images/aai_presentation_last_slide.svg" alt="Snow" style="width:100%;">